In [ ]:
import carla
import math
import random
import time
import numpy as np
import inspect
import cv2

#riferimento al tutorial
#https://carla.readthedocs.io/en/latest/tuto_first_steps/
#https://www.youtube.com/watch?v=pONr1R1dy88

#size di default del sensore
IM_WIDTH = 500
IM_HEIGHT = 500

#cattura i raw data del sensore e li trasforma in immagini priettate in imgshow di opencv
def process_img(image):
    i = np.array(image.raw_data) #raw.data è un array flat
    #print(i.shape)
    i2 = i.reshape((IM_HEIGHT, IM_WIDTH, 4))
    i3 = i2[:, :, :3] #intera altezza, larghezza, i primi tre valori sono rgb
    #print(i3)
    cv2.imshow("RGB_SENS",i3) #NON DARE UN TITOLO SE NO NON FUNZIONA
    cv2.waitKey(1) #ms: aggiornamento finestra a 0 non funziona
    return i3/255.0 #normalizzazione valori tra 0 e 1

#lista attori creati nel simulatore
actors = []

client = carla.Client('localhost', 2000)
#accesso agli asset della simulazione
world = client.get_world()

#per cambiare mappa, richiede compilazione shader se non giò fatta
#python3 config.py --map Town10HD #default
#python3 config.py --map Town07 #piccola

#serve  per creare oggetti nella simulazione tipo veicoli, caos, pedoni.
bp_lib = world.get_blueprint_library()

#punti per spawnare oggetti
spawn_points = world.get_map().get_spawn_points();

vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020')
#il try serve per trovare locazioni libere altrimenti fallirebbe
#mosta il veicolo in una posizione casuale tra 200 disponibili
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))
actors.append(vehicle)

#posizionamento iniziale telecamera
spectator = world.get_spectator()
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),vehicle.get_transform().rotation)
spectator.set_transform(transform)


#settaggio di sensori (gps, collisioni ecc..), in questo caso camera rgb
camera_bp = bp_lib.find('sensor.camera.rgb')
#ATTRIBUTI PER APPRENDIMENTO
camera_bp.set_attribute("fov","110")
camera_bp.set_attribute('image_size_x', f"{IM_WIDTH}")
camera_bp.set_attribute('image_size_y', f"{IM_HEIGHT}")
#camera_bp.set_attribute('sensor_tick', '0.5') #rilevazione ogni millesimo di secondo

#print(inspect.getmembers(camera_bp))

camera_init_trans = carla.Transform(carla.Location(x=2.5, z=0.7)) #posizione telecamera
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)
actors.append(camera);



cv2.destroyAllWindows()
camera.listen(lambda data: process_img(data))

vehicle.set_autopilot(True)

#QUESTO PER SEGUIRE IL VEICOLo ogni 5 secondi
while True:
    time.sleep(5)
    spectator.set_transform(camera.get_transform())

#macchina guida dritto
#vehicle.apply_control(carla.VehicleControl(throttle=1.0, steer=0.0))





In [1]:
for a in actors:
    a.destroy()
cv2.destroyAllWindows()
#for v in world.get_actors().filter('*vehicle*'):
#    v.destroy()

NameError: name 'actors' is not defined